In [1]:
import cv2
import smtplib
from email.mime.text import MIMEText
import time
import psutil
import os
from ultralytics import YOLO
from dotenv import load_dotenv
from datetime import datetime
import numpy as np


In [2]:
video_path = 'http://10.1.18.177:8080/video'
out_path = r'output10.mp4'

# Get process ID
process = psutil.Process(os.getpid())

# Load YOLO model (Nano version for speed)
model = YOLO("yolov8n.pt", verbose=False)

# Define video path
cap = cv2.VideoCapture(video_path)
caps = [
    cv2.VideoCapture('https://10.1.18.177:8080/video'), 
    cv2.VideoCapture('https://10.1.16.95:8080/video')
]
cams = ['haridas', 'ronak']

# Email configuration
load_dotenv(dotenv_path=r'C:\Users\haridas\Desktop\PU\HT\smart-watch\smart-watch\.env')
sender_email = os.getenv('SENDER_EMAIL')
receiver_email = os.getenv('RECEIVER_EMAIL')
smtp_server = "smtp.gmail.com"
smtp_port = 587
email_password = os.getenv("SENDER_PASS")

# Cooldown period (in seconds) for email alerts
notification_cooldown_period = 300
last_trespass_alert_time = 0
last_trespass_alert_times = [0] * len(caps)

# Motion detection variables
has_motion = False
has_motions = [False] * len(caps)
last_motion_at = time.time()
last_motion_ats = [0] * len(caps)
check_period = 5

# Background subtractor for motion detection
backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=16, detectShadows=False)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# FPS control
fps = 10
i = 1

# video saving control
fourcc = cv2.VideoWriter_fourcc(*'AVC1')  # Codec for MP4 format
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(out_path, fourcc, FPS, (frame_width, frame_height))
# print(frame_width, frame_height, FPS)




In [3]:
def add_text(frame, text):
    """Adds text to the top-left corner of a frame."""
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    return frame

def arrange_frames(frames, grid_size=(2, 2), frame_size=(320, 240)):
    """Arrange frames in a grid layout."""
    rows, cols = grid_size
    blank_image = np.zeros((rows * frame_size[1], cols * frame_size[0], 3), dtype=np.uint8)

    for idx, frame in enumerate(frames):
        if frame is None:
            # Replace None frame with a blank frame
            frame = np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8)
        resized_frame = cv2.resize(frame, frame_size)
        labeled_frame = add_text(resized_frame, f"Frame {idx}")

        row, col = divmod(idx, cols)
        y_start, y_end = row * frame_size[1], (row + 1) * frame_size[1]
        x_start, x_end = col * frame_size[0], (col + 1) * frame_size[0]
        blank_image[y_start:y_end, x_start:x_end] = labeled_frame

    return blank_image

def send_email_alert(timestamp):
    subject = "Trespassing Alert"
    body = f"A human trespassing event was detected at [Camera_ID] on {timestamp}."

    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, email_password)
        server.sendmail(sender_email, [receiver_email], msg.as_string())
        server.quit()
        print(f"Alert email sent at {timestamp}.")
    except Exception as e:
        print("Failed to send email:", e)


In [4]:
isEnd = False
frames = [0] * len(caps)

while not isEnd:
    for idx, cap in enumerate(caps):
        ret, frames[idx] = cap.read()
        if not ret:
            break

        if has_motions[idx]:
            if i % fps != 0:
                i += 1
                continue
            i = 1

            # YOLO human detection
            results = model.predict(frames[idx], verbose=False)
            human_detected = False

            for result in results:
                for box in result.boxes:
                    if int(box.cls.item()) == 0 and box.conf >= 0.5:  # Confidence threshold
                        human_detected = True
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cv2.rectangle(frames[idx], (x1, y1), (x2, y2), (0, 0, 255), 2)
                        
            if human_detected and time.time() - last_trespass_alert_times[idx] >= notification_cooldown_period:
                last_trespass_alert_times[idx] = time.time()
                detection_time_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
                print("Detected Trespassing at " + detection_time_str)
                # send_email_alert(detection_time_str)  # Pass timestamp to email function

            if time.time() - last_motion_ats[idx] > check_period:
                has_motions[idx] = False
                last_motion_ats[idx] = time.time()
        else:
            human_detected = False
            fgMask = backSub.apply(frames[idx])
            fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
            contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) > 500:
                    x, y, w, h = cv2.boundingRect(contour)
                    aspect_ratio = h / float(w)
                    if aspect_ratio > 1.2:
                        cv2.rectangle(frames[idx], (x, y), (x+w, y+h), (0, 255, 0), 2)
                        has_motions[idx] = True
                        last_motion_ats[idx] = time.time()
                        break
        if cv2.waitKey(1) == ord("q"):
            isEnd = True
        frames[idx] = add_text(frames[idx], cams[idx])
    cv2.imshow("Frame", arrange_frames(frames))
    out.write(frames[idx])  # Save video
    if cv2.waitKey(1) == ord("q"):
        isEnd = True

for cap in caps:
    cap.release()
    
out.release()
cv2.destroyAllWindows()

# Get memory usage in MB
memory_used = process.memory_info().rss / (1024 * 1024)

print(f"Peak Memory Usage: {memory_used:.2f} MB")


Detected Trespassing at 2025-03-31 00:44:54
Detected Trespassing at 2025-03-31 00:44:56
Peak Memory Usage: 733.65 MB


In [ ]:
# Pi + Coral Edge TPU: Best choice if we can use Google’s Edge TPU for detection instead of YOLO.
